# Chest X-Ray Images (Pneumonia)
Build a 

In [13]:
# Essential Imports
import tensorflow as tf
from tensorflow import keras as k
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print(tf.__version__)

2.4.0-dev20200812
